In [1]:
!pip install transformers

In [2]:

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import shutil
import sys   

In [3]:
train_path = "/kaggle/input/data-capec-multilabel/capec_mutilabel_new.csv"
test_path = "/kaggle/input/data-capec-multilabel/capec_mutilabel_new.csv"

In [4]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

/tmp/ipykernel_26/897552558.py:1: DtypeWarning: Columns (5,6,22) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(train_path)
/tmp/ipykernel_26/897552558.py:2: DtypeWarning: Columns (5,6,22) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv(test_path)


In [5]:
train_df.head()

,text,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,request_http_method,request_http_request,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,126 - Path Traversal,66 - SQL Injection,16 - Dictionary-based Password Attack,310 - Scanning for Vulnerable Software,153 - Input Data Manipulation,248 - Command Injection,274 - HTTP Verb Tampering,194 - Fake the Source of Data,34 - HTTP Response Splitting,33 - HTTP Request Smuggling
0,GET /,NaN,NaN,NaN,NaN,GET,/,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,GET /blog/index.php/2020/04/04/voluptatum-repr...,NaN,NaN,NaN,NaN,GET,/blog/index.php/2020/04/04/voluptatum-reprehen...,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,GET /blog/xmlrpc.php?rsd,NaN,NaN,NaN,NaN,GET,/blog/xmlrpc.php?rsd,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,GET /,NaN,NaN,NaN,NaN,GET,/,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,GET /blog/index.php/2020/04/04/nihil-tenetur-e...,NaN,NaN,NaN,NaN,GET,/blog/index.php/2020/04/04/nihil-tenetur-et-ad...,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
train_df.drop(train_df.columns[1:24], axis=1, inplace=True)

In [7]:
df2 = train_df.loc[200000:624245]
df2 = df2.loc[(df2['242 - Code Injection'] == 1) 
              | (df2['16 - Dictionary-based Password Attack']==1)
              | (df2['153 - Input Data Manipulation']==1)
              | (df2['248 - Command Injection']==1)
              | (df2['34 - HTTP Response Splitting']==1)
              | (df2['274 - HTTP Verb Tampering'] == 1)
              | (df2['33 - HTTP Request Smuggling']==1)]

In [8]:
train_df = pd.concat([train_df.head(200000), df2], ignore_index=True)

In [9]:
train_df.columns

Index(['text', '000 - Normal', '272 - Protocol Manipulation',
       '242 - Code Injection', '88 - OS Command Injection',
       '126 - Path Traversal', '66 - SQL Injection',
       '16 - Dictionary-based Password Attack',
       '310 - Scanning for Vulnerable Software',
       '153 - Input Data Manipulation', '248 - Command Injection',
       '274 - HTTP Verb Tampering', '194 - Fake the Source of Data',
       '34 - HTTP Response Splitting', '33 - HTTP Request Smuggling'],
      dtype='object')

In [10]:
target_list = ["000 - Normal", "272 - Protocol Manipulation", "242 - Code Injection", 
               "88 - OS Command Injection", "126 - Path Traversal", "66 - SQL Injection",
               "16 - Dictionary-based Password Attack", "310 - Scanning for Vulnerable Software", 
               "153 - Input Data Manipulation", "248 - Command Injection", 
               "274 - HTTP Verb Tampering", "194 - Fake the Source of Data", 
               "34 - HTTP Response Splitting", "33 - HTTP Request Smuggling"]

In [11]:
len(target_list)

14

In [12]:
# hyperparameters
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 2
LEARNING_RATE = 1e-05

In [13]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
from transformers import BertTokenizer, BertModel

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [14]:
tokenizer = AutoTokenizer.from_pretrained('jackaduma/SecBERT')


In [15]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['text']
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [16]:
train_size = 0.8
train_df1 = train_df.sample(frac=train_size, random_state=200).reset_index(drop=True)
val_df = train_df.drop(train_df1.index).reset_index(drop=True)

In [17]:
train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)

In [18]:
train_data_loader = torch.utils.data.DataLoader(train_dataset, 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset, 
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [19]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [20]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [21]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = AutoModel.from_pretrained('jackaduma/SecBERT')
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 14)
    
    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids, 
            attention_mask=attn_mask, 
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()

model.to(device)

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=0)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [22]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [23]:
val_targets=[]
val_outputs=[]

In [24]:
def train_model(n_epochs, training_loader, validation_loader, model, 
                optimizer, checkpoint_path, best_model_path):
   
  # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf
   
 
    for epoch in range(1, n_epochs+1):
        train_loss = 0
        valid_loss = 0
        model.train()
        print('############# Epoch {}: Training Start   #############'.format(epoch))
        for batch_idx, data in enumerate(training_loader):
            #print('yyy epoch', batch_idx)
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs = model(ids, mask, token_type_ids)

            optimizer.zero_grad()
            loss = loss_fn(outputs, targets)
            #if batch_idx%5000==0:
             #   print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #print('before loss data in training', loss.item(), train_loss)
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
            #print('after loss data in training', loss.item(), train_loss)
            
    print('############# Epoch {}: Training End     #############'.format(epoch))
    
    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################    
    # validate the model #
    ######################
 
    model.eval()
   
    with torch.no_grad():
        for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

        print('############# Epoch {}: Validation End     #############'.format(epoch))
        # calculate average losses
        #print('before cal avg train loss', train_loss)
        if len(training_loader) > 0:
            train_loss = train_loss/len(training_loader)
        if len(validation_loader) > 0:
            valid_loss = valid_loss/len(validation_loader)
        # print training/validation statistics 
        print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))

        # create checkpoint variable and add important data
        checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }

        # save checkpoint
        save_ckp(checkpoint, False, checkpoint_path, best_model_path)

        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
            # save checkpoint as best model
            save_ckp(checkpoint, True, checkpoint_path, best_model_path)
            valid_loss_min = valid_loss

        print('############# Epoch {}  Done   #############\n'.format(epoch))

        return model

In [25]:
ckpt_path = "/kaggle/working/curr_ckpt"
best_model_path = "/kaggle/working/best_model.pt"

In [26]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path)

############# Epoch 1: Training Start   #############
############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Avgerage Training Loss: 0.000002 	Average Validation Loss: 0.000012
Validation loss decreased (inf --> 0.000012).  Saving model ...
############# Epoch 2  Done   #############



In [27]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report,accuracy_score, f1_score

In [28]:
# testing
true_labels = list()
predicted_labels = list()

check = 1
num_correct = 0;
num_sample = 0
for i in range(200000, 600000, 1):
    check = 1
    example = test_df['text'][i]
    num_sample += 1
    encodings = tokenizer.encode_plus(
        example,
        None,
        add_special_tokens=True,
        max_length=MAX_LEN,
        padding='max_length',
        return_token_type_ids=True,
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    model.eval()
    with torch.no_grad():
        input_ids = encodings['input_ids'].to(device, dtype=torch.long)
        attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
        token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
        output = model(input_ids, attention_mask, token_type_ids)
        final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
    #     print(final_output)
        for x in final_output:
            true_label = list()
            predicted_label = list()
            for j, y in enumerate(x):
#     #             print(i)
#                 if y < 0.5: and test_df[target_list[j]][i] == 1:
# #                     check = 0
# #                     break
#                 elif y > 0.5 and test_df[target_list[j]][i] == 0:
# #                     check = 0
# #                     break
# #         if check == 1:
# #             num_correct += 1
                if y < 0.5:
                    predicted_label.append(0)
                else:
                    predicted_label.append(1)
                if test_df[target_list[j]][i] == 1:
                    true_label.append(1)
                else:
                    true_label.append(0)
#             print(predicted_label)
            predicted_labels.append(predicted_label)
            true_labels.append(true_label)
# print(predicted_labels)
# print("true label: ", true_labels)
# acc = num_correct / num_sample
# print(acc)
score = f1_score(true_labels,predicted_labels,average="micro")
precision = precision_score(true_labels, predicted_labels,average="micro")
recall = recall_score(true_labels, predicted_labels,average="micro")
report = classification_report(true_labels,predicted_labels,digits=4)
acc= accuracy_score(true_labels, predicted_labels)
#classifaction_report_csv(report,precision,recall,score,0)
print ('\n clasification report:\n', report)
print ('F1 score:', score)
print ('Recall:', recall)
print ('Precision:', precision)
print ('Acc:', acc)



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 clasification report:
               precision    recall  f1-score   support

           0     0.9134    0.7601    0.8297    167414
           1     0.9630    0.1867    0.3128      8093
           2     0.8045    0.9564    0.8739     14925
           3     0.8560    0.2132    0.3414      6298
           4     0.9379    0.4643    0.6211      3515
           5     0.8362    0.9393    0.8848    177488
           6     0.5917    0.9930    0.7415      1436
           7     0.0000    0.0000    0.0000         3
           8     0.0000    0.0000    0.0000      1023
           9     0.0000    0.0000    0.0000         0
          10     0.6051    0.9996    0.7538      5436
          11     0.4369    0.0833    0.1399      1081
          12     0.9905    0.9661    0.9782     19738
          13     0.8849    0.9660    0.9237      1059

   micro avg     0.8625    0.8338    0.8479    407509
   macro avg     0.6300    0.5377    0.5286    407509
weighted avg     0.8710    0.8338    0.8379    407509
 

In [29]:
# from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report,accuracy_score, f1_score

# def evaluate(model, percentage = 80):
#     with torch.no_grad():
#         predicted_labels_dict = {
#           0: 0,
#           1: 0,
#           2: 0,
#           3: 0,
#           4: 0,
#           5: 0,
#           6: 0,
#         }
#         model.eval()
#         dev_df = pd.read_csv(test_path)
#         data_size = dev_df.shape[0]
#         selected_for_evaluation = int(data_size*percentage/100)
#         dev_df = dev_df.head(selected_for_evaluation)
#         dataset = ReviewDataset(dev_df)
#         dataloader = DataLoader(dataset = dataset, batch_size = training_parameters["batch_size"], shuffle = True, num_workers = 2)

#         true_labels = list()
#         predicted_label = list()
#         for input_ids, attention_mask, token_type_ids, labels in dataloader:
#             inputs = {
#                 "input_ids": input_ids.squeeze(axis=1),
#                 "attention_mask": attention_mask.squeeze(axis=1),
#                 "token_type_ids" : token_type_ids.squeeze(axis=1),
#                 "labels": labels,
#             }
#             for k, v in inputs.items():
#                 inputs[k] = v.to(device)
#             attack_pred, _ = model(**inputs)
#             predicted_label.extend(attack_pred.max(dim = 1)[1].cpu().numpy())
#             true_labels.extend(inputs['labels'].cpu().numpy())
#             _, predicted_labels = compute_accuracy(attack_pred, inputs["labels"])

#             for i in range(7):
#                 predicted_labels_dict[i] += predicted_labels[i]

#         score = f1_score(true_labels,predicted_label,average="macro")
#         precision = precision_score(true_labels, predicted_label,average="macro")
#         recall = recall_score(true_labels, predicted_label,average="macro")
#         report = classification_report(true_labels,predicted_label,digits=4)
#         acc= accuracy_score(true_labels, predicted_label)
#         #classifaction_report_csv(report,precision,recall,score,0)
#         print ('\n clasification report:\n', report)
#         print ('F1 score:', score)
#         print ('Recall:', recall)
#         print ('Precision:', precision)
#         print ('Acc:', acc)
#         print('Confusion Matrix: \n',confusion_matrix(true_labels, predicted_label))
#         print(predicted_labels_dict)

In [30]:
def accuracy(y_true, y_pred):
    temp = 0
    for i in range(len(y_true)):
        numerator = sum(np.logical_and(y_true[i], y_pred[i]))
        denominator = sum(np.logical_or(y_true[i], y_pred[i]))
        if denominator != 0:
            temp += numerator / denominator
    return temp / len(y_true)

In [31]:
acc_ = accuracy(true_labels,predicted_labels)
print("accuracy:", acc_)

accuracy: 0.8375585416666664
